In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1,PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_metric,load_dataset,Dataset

import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer


import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
from tqdm.auto import tqdm, trange

import csv
import gc

from src.utils.myutils import clean_memory,compute_metrics,preprocess_data

model_checkpoint = 'roberta-base'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
transformers.utils.logging.set_verbosity_error()
BATCH_SIZE = 16

## Data preprocessing

In [2]:
data = load_dataset('csv',data_files=PATH+"/data/EN/processed/BABE/babe_sg2.csv")['train']

Using custom data configuration default-9aed32b2774fea6c
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-9aed32b2774fea6c/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [3]:
data = data.train_test_split(0.2)
data

Loading cached split indices for dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-9aed32b2774fea6c/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-e14ec89e5ba972e8.arrow and /home/horyctom/.cache/huggingface/datasets/csv/default-9aed32b2774fea6c/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-46b43f2365d207ce.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2938
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 735
    })
})

In [8]:
data_train = data['train']
data_test = data['test']

## Training

In [9]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint);
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
model.to(device);

In [11]:
training_args = TrainingArguments(
    output_dir='../',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    warmup_steps=50,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.05)

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
tokenized_train = preprocess_data(data_train,tokenizer,'text')
tokenized_test = preprocess_data(data_test,tokenizer,'text')

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### 5-fold CV

In [14]:
f1_scores = []

In [16]:
for train_index, val_index in skfold.split(data_train['text'],data_train['label']):
    
    token_train = Dataset.from_dict(tokenized_train[train_index])
    token_valid = Dataset.from_dict(tokenized_train[val_index])
    
    model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                      tokenizer=tokenizer)
    trainer.train()
    
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    f1_scores.append(compute_metrics(model,device,eval_dataloader)['f1'])


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/torch_formatter.py:44: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:135.)
  return torch.tensor(value, **{**default_dtyp

Step,Training Loss
50,0.653600
100,0.573300
150,0.493500
200,0.367500
250,0.376600
300,0.339900
350,0.230700
400,0.203700




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

Step,Training Loss
50,0.653100
100,0.552400
150,0.520600
200,0.425400
250,0.409500
300,0.358400
350,0.241700
400,0.300300




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

...



Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

Step,Training Loss
50,0.660800
100,0.570300
150,0.523800
200,0.437600
250,0.436400
300,0.403500
350,0.269900
400,0.238000




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  

Step,Training Loss
50,0.664200
100,0.580800
150,0.539200
200,0.452700
250,0.412900
300,0.383200
350,0.255700
400,0.257900




Training completed. Do not forget to share your model on huggingface.co/models =)




In [19]:
np.mean(f1_scores)

0.8110993289990611

In [20]:
test_dataloader = DataLoader(tokenized_test, batch_size=BATCH_SIZE, collate_fn=data_collator)

In [25]:
print(data['train']['text'][0])

Like many Americans also crippled by student loans, Brewster’s been making sacrifices since she graduated: She put off her wedding for two years and once she and her husband, Ryan, finally got married in November 2018, they decided they couldn’t afford a honeymoon.


In [28]:
compute_metrics(model,device,test_dataloader)['f1']

0.8149659863945579

### Inferrence experiments

In [59]:
sentence = 'Orange Is the New Black" star Yael Stone is renouncing her U.S. green card to return to her native Australia in order to fight climate change.'
#sentence = 'This might be biased but mustache suits you.'
toksentence = tokenizer(sentence,truncation=True,return_tensors="pt")
model.eval()
with torch.no_grad():
    toksentence.to(device)
    output = model(**toksentence)

In [60]:
classification = F.softmax(output.logits,dim=1).argmax(dim=1)
print(sentence,': ',{0:'unbiased',1:'biased'}[classification[0].item()])

Orange Is the New Black" star Yael Stone is renouncing her U.S. green card to return to her native Australia in order to fight climate change. :  unbiased
